In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
# sigmoid.py (https://github.com/fpelogia/epid-models/blob/master/sigmoid.py)
from sigmoid import f_t, deriv_f_t 
# new_wave.py (https://github.com/fpelogia/epid-models/blob/master/new_wave.py)
from new_wave import new_wave_detection, filter_data, forward_euler 

In [2]:
# Import data
data = pd.read_csv("../Datasets/osaka.csv") 
city_name = 'Osaka' 
indicator='cases'

In [3]:
if(indicator == 'deaths'):
    #deaths
    acc_data = data.cumulative_deceased
else:
    # cases
    #acc_data = data.total_confirmed
    acc_data = data.cumulative_confirmed #(Japan)

normalized_acc_data = acc_data / max(acc_data)
t = np.linspace(0, len(acc_data)-1, len(acc_data))

normalized_acc_data = normalized_acc_data.tolist()

if(indicator == 'deaths'):
    daily_data = data.new_deceased
else:
    daily_data = data.new_confirmed


scaling_factor = 500
acc_data = acc_data / scaling_factor
daily_data = list(daily_data/ scaling_factor)

In [5]:
n_sig = 1
sig_params = []
def model(t, A, tp, delta, nu):
    res = np.zeros(n_days)
    for i in range(n_sig - 1):
        [A_i, tp_i, delta_i, nu_i] = sig_params[i]
        res += f_t(t[:n_days], A_i, tp_i, delta_i, nu_i)

    res += f_t(t[:n_days], A, tp, delta, nu)
    return res

def model_daily(t, A, tp, delta, nu):
    res = np.zeros(n_days)
    for i in range(n_sig - 1):
        [A_i, tp_i, delta_i, nu_i] = sig_params[i]
        res += deriv_f_t(t[:n_days], A_i, tp_i, delta_i, nu_i)

    res += deriv_f_t(t[:n_days], A, tp, delta, nu)
    return res

# Integral Time Square Error (ITSE)
def ITSE(x):
    # model parameters
    A = x[0]
    tp = x[1]
    delta = x[2]
    nu = x[3]

    y_t = acc_data[:n_days]
    y_m = model(t[:n_days], A, tp, delta, nu)
    return np.sum(t[:n_days]*(y_t - y_m)**2)

# Mean Squared Error (MSE)
def MSE(x):
    # model parameters
    A = x[0]
    tp = x[1]
    delta = x[2]
    nu = x[3]

    y_t = acc_data[:n_days]
    y_m = model(t[:n_days], A, tp, delta, nu)
    return (1/len(y_t))*np.sum((y_t - y_m)**2)

def loss_f(x, lf):
    if(lf == 'MSE'):
        return MSE(x)
    elif(lf == 'ITSE'):
        return ITSE(x)
    else:
        return MSE(x)


def loss_f_sym(x, lf):
    # nu = 1 (symmetric sigmoid)
    if(lf == 'MSE'):
        return MSE([x[0], x[1], x[2], 1]) 
    elif(lf == 'ITSE'):
        return ITSE([x[0], x[1], x[2], 1]) 
    else:
        return MSE([x[0], x[1], x[2], 1]) 
    

# Inequality contraints need to return f(x), where f(x) >= 0
def constr1(x):
    # A >= 0
    return x[0]
def constr2(x):
    # tp >= 0
    return x[1]
def constr3(x):
    # delta = 0.1
    return x[2] - 1e-1
def constr4(x):
    # nu > 0.1
    return x[3] - 1e-1

con1 = {'type':'ineq', 'fun':constr1}
con2 = {'type':'ineq', 'fun':constr2}
con3 = {'type':'ineq', 'fun':constr3}
con4 = {'type':'ineq', 'fun':constr4}     
cons = [con1, con2, con3, con4] 

In [ ]:
# n_sig = 1
# sig_params = []
# for i in range(len(x_nw) + 1):
#     if(i == len(x_nw)):
#         n_days = len(acc_data)
#     else:
#         n_days = x_nw[i]
#     print(f'========= Wave nr {i + 1} =========')
#     print('From 0 to ', n_days)
#     print('Step 1')
#     # Step 1 - Optimize a symmetric sigmoid (nu = 1)
#     # Initial values
#     if(i == 0):
#         y_t = acc_data[:n_days]
#         A0 = 2*max(y_t)
#         tp0 = (2/3)*len(y_t)
#         delta0 = (1/4)*len(y_t)
#         nu0 = 1
#     else:
#         tp0 += 90
#         A0 *= 0.05
#     # else:
#     #     [A0, tp0, delta0, nu0] = sig_params[len(sig_params) - 1]
#     #     tp0 = tp0 + 100
#     x0 = [A0, tp0, delta0, nu0]
#     sol = minimize(loss_f_sym, x0, constraints=cons, args=('MSE'), method='SLSQP')
#     print(sol)

#     # Optimal values
#     [A, tp, delta, nu] = sol.x

#     print('Step 2')
#     # Step 2 - Optimize an assymmetric sigmoid
#     # using optimal values of step 1 as the starting point
#     [A0, tp0, delta0, nu0] = sol.x

#     x0 = [A0, tp0, delta0, nu0]
#     sol = minimize(loss_f, x0, constraints=cons, args=('MSE'), method='SLSQP')
#     print(sol)

#     # Optimal values
#     [A, tp, delta, nu] = sol.x

#     y_t = acc_data[:n_days]
#     y_m0 = model(t[:n_days], A0, tp0, delta0, nu0)
#     y_m = model(t[:n_days], A, tp, delta, nu)
#     s = "" if (n_sig == 1) else "s"
#     plt.figure(figsize=(10,6))
#     plt.title(f'{city_name} - Model with {n_sig} sigmoid{s}')
#     plt.plot(y_t, label='$y_t$ (target)')
#     plt.plot(y_m0, label='$y_m0$ (model after Step 1)')
#     plt.plot(y_m, label='$y_m$ (model after Step 2)')
#     plt.xlabel('t (days)')
#     plt.ylabel('acc. number of deaths')
#     plt.legend()
#     #plt.savefig(f'output/{i:003}', facecolor='white', dpi=100)
#     plt.show()
    
#     n_sig += 1
#     sig_params.append([A, tp, delta, nu])
#     print(f'===================================')    

In [ ]:
# n_sig = 5

# n_weeks_pred = 2
# y_t = acc_data[:n_days]
# y_m_daily = model_daily(t[:n_days], A, tp, delta, nu)
# daily_data = forward_euler(t[:n_days], 1, acc_data[:n_days], 0)
# s = "" if (n_sig == 1) else "s"

# # Plotting Model vs Data
# plt.figure(figsize=(10,6))
# fig, ax = plt.subplots(figsize=[10,6])
# #plt.xlim(x_nw[i] - 7*(n_weeks_pred + 1), x_nw[i])
# ax.set_title(f'{city_name} - Model x Data - {n_sig} sigmoid{s}')
# ax.scatter(t[:n_days], acc_data[:n_days], label='Data', c='gray')
# ax.vlines(n_days - 7*n_weeks_pred, 0, max(acc_data[:n_days]), colors='dimgray', linestyles='dashdot', zorder=1, label=f"last {7*n_weeks_pred} days")
# ax.plot(y_m, label='Model after fitting', c='r')
# ax.set_xlabel('t (days)')
# ax.set_ylabel('acc. number of deaths')
# ax.legend()
# X_detail = t[n_days - 7*n_weeks_pred: n_days]
# Y_detail = y_m[n_days - 7*n_weeks_pred: n_days]
# # detail prediction
# # if(n_sig < 3):
# #     sub_axes = ax.inset_axes([.17, .45, .25, .35]) 
# # else:
# #     sub_axes = ax.inset_axes([.6, .15, .25, .25]) 
# #sub_axes.scatter(t[n_days - 7*n_weeks_pred:n_days], acc_data[n_days - 7*n_weeks_pred:n_days], label='Data', c='gray')
# #sub_axes.plot(X_detail, Y_detail, c = 'r') 
# #sub_axes.set_xticks(X_detail[0::3])
# #ax.indicate_inset_zoom(sub_axes, edgecolor="black")    
# plt.savefig(f'output/005.png', facecolor='white', dpi=100)
# plt.show()

# # DAILY DATA
# plt.figure(figsize=(10, 6))

# plt.title(f'{city_name} - Model x Daily data')
# plt.plot(daily_data[:n_days], label="daily data", c='gray', lw=0.8, linestyle='dashed')
# plt.plot(y_m_daily, label='model after fitting (daily)', c='r')
# plt.vlines(n_days - 7*n_weeks_pred, 0, max(daily_data[:n_days]), colors='dimgray', linestyles='dashdot', zorder=1, label=f"last {7*n_weeks_pred} days")
# plt.xlabel('t (days)')
# plt.ylabel('daily number of deaths')
# plt.legend(loc=2) # upper left
# plt.savefig(f'output/Daily_{city_name}_{n_sig}_wave', facecolor='white', dpi=100)
# plt.show()

In [39]:
x_nw = list(np.array(x_nw) + 14)

In [ ]:
# n_weeks_pred = 2
# n_sig = 1
# sig_params = []

# for i in range(len(x_nw)):
#     n_days = x_nw[i]- 7*n_weeks_pred
#     print(f'========= Wave nr {i + 1} =========')
#     print('From 0 to ', n_days)
#     print('Step 1')
#     # Step 1 - Optimize a symmetric sigmoid (nu = 1)
#     # Initial values
#     if(i == 0):
#         y_t = acc_data[:n_days]
#         A0 = 2*max(y_t)
#         tp0 = (2/3)*len(y_t)
#         delta0 = (1/4)*len(y_t)
#         nu0 = 1
#     else:
#         tp0 += 90
#         A0 *= 0.05

#     x0 = [A0, tp0, delta0, nu0]
#     sol = minimize(loss_f_sym, x0, constraints=cons, args=('MSE'), method='SLSQP')
#     print(sol)

#     # Optimal values
#     [A, tp, delta, nu] = sol.x

#     print('Step 2')
#     # Step 2 - Optimize an assymmetric sigmoid
#     # using optimal values of step 1 as the starting point
#     [A0, tp0, delta0, nu0] = sol.x

#     x0 = [A0, tp0, delta0, nu0]
#     sol = minimize(loss_f, x0, constraints=cons, args=('MSE'), method='SLSQP')
#     print(sol)

#     # Optimal values
#     [A, tp, delta, nu] = sol.x

#     # due to filtering delay
#     n_days = x_nw[i]

#     y_t = acc_data[:n_days]
#     y_m0 = model(t[:n_days], A0, tp0, delta0, nu0)
#     y_m = model(t[:n_days], A, tp, delta, nu)
#     y_m_daily = model_daily(t[:n_days], A, tp, delta, nu)
#     s = "" if (n_sig == 1) else "s"

#     # Plotting Model vs Data

#     fig, axs = plt.subplots(1, 2, figsize=[15,5])
#     #plt.xlim(x_nw[i] - 7*(n_weeks_pred + 1), x_nw[i])
#     axs[0].set_title(f'{city_name} - Model x Data - {n_sig} sigmoid{s}')
#     axs[0].scatter(t[:n_days], acc_data[:n_days], label='Data', c='gray')
#     axs[0].vlines(n_days - 7*n_weeks_pred, 0, max(acc_data[:n_days]), colors='dimgray', linestyles='dashdot', zorder=1, label=f"Last {7*n_weeks_pred} days")
#     axs[0].plot(y_m, label='Model', c='r')
#     axs[0].set_xlabel('t (days)')
#     axs[0].set_ylabel('acc. number of deaths')
#     X_detail = t[n_days - 7*n_weeks_pred: n_days]
#     Y_detail = y_m[n_days - 7*n_weeks_pred: n_days]
#     # detail prediction
#     if(n_sig < 3):
#         sub_axes = axs[0].inset_axes([.17, .45, .25, .35]) 
#     else:
#         sub_axes = axs[0].inset_axes([.6, .15, .25, .25]) 
#     sub_axes.scatter(t[n_days - 7*n_weeks_pred:n_days], acc_data[n_days - 7*n_weeks_pred:n_days], label='Data', c='gray')
#     sub_axes.plot(X_detail, Y_detail, c = 'r') 
#     #sub_axes.set_xticks(X_detail[0::3])
#     axs[0].indicate_inset_zoom(sub_axes, edgecolor="black")    
#     #plt.savefig(f'output/Acc_{city_name}_{n_sig}_sig', facecolor='white', dpi=100)


#     # Plotting Daily Data

#     axs[1].set_title(f'{city_name} - Model x Daily data')
#     axs[1].plot(daily_data[:n_days], label="Data", c='gray', lw=0.8, linestyle='dashed')
#     axs[1].plot(y_m_daily, label='Model', c='r')
#     axs[1].vlines(n_days - 7*n_weeks_pred, 0, max(daily_data[:n_days]), colors='dimgray', linestyles='dashdot', zorder=1, label=f"Last {7*n_weeks_pred} days")
#     axs[1].set_xlabel('t (days)')
#     axs[1].set_ylabel('daily number of deaths')
#     axs[1].legend(loc=2) # upper left
#     plt.savefig(f'CLAIB/ACC_DAILY_{city_name}_{n_sig}_wave', facecolor='white', dpi=100)
#     plt.show()
    
#     n_sig += 1
#     sig_params.append([A, tp, delta, nu])
#     print(f'===================================')    

# plt.tight_layout()
# #plt.savefig(f'output/Daily_{city_name}_2w_pred', facecolor='white', dpi=100)
# plt.show()


#### ===== Rascunho (testes para melhor entendimento) =====